# Import

In [1]:
import pandas as pd

# Data load

In [2]:
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

In [3]:
train = train[:int(len(train)*0.01)]

In [16]:
# Function to calculate VIF
import statsmodels.api as sm

def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

# Pycaret

In [45]:
from audioop import minmax
from pycaret.regression import *

exp_clf = setup(data = train, target = 'target', session_id = 123, normalize = True, train_size = 0.8, use_gpu = True, data_split_shuffle = True, verbose = True, n_jobs = -1, normalize_method='minmax')

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Regression
3,Data shape,"(47012, 28)"
4,Train data shape,"(37609, 28)"
5,Test data shape,"(9403, 28)"
6,Numeric features,28
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


In [46]:
compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,4.8154,44.6525,6.6815,0.8244,0.2030,0.1497,2.1630
et,Extra Trees Regressor,5.0750,50.2152,7.0856,0.8025,0.2138,0.1568,1.6590
lightgbm,Light Gradient Boosting Machine,5.5289,52.0464,7.2138,0.7953,0.2216,0.1760,1.3680
dt,Decision Tree Regressor,6.3676,81.4114,9.0219,0.6800,0.2748,0.1922,0.3270
gbr,Gradient Boosting Regressor,6.9820,79.4317,8.9121,0.6877,0.2648,0.2233,4.8630
knn,K Neighbors Regressor,7.1359,88.1164,9.3865,0.6534,0.2725,0.2230,2.9780
ada,AdaBoost Regressor,8.8347,117.4570,10.8374,0.5382,0.3073,0.2757,1.6090
huber,Huber Regressor,9.9022,151.0226,12.2885,0.4063,0.3449,0.3081,1.2360
br,Bayesian Ridge,9.9285,150.5119,12.2677,0.4083,0.3429,0.3100,0.0980
ridge,Ridge Regression,9.9288,150.5178,12.2680,0.4083,0.3429,0.3100,0.0920


RandomForestRegressor(n_jobs=-1, random_state=123)

In [4]:
lgbm = create_model('lightgbm')
rf = create_model('rf')

tuned_lgbm = tune_model(lgbm, optimize = 'MAE', choose_better=True, n_iter=30, search_library='optuna')
tuned_rf = tune_model(rf, optimize = 'MAE', choose_better=True, n_iter=30, search_library='optuna')

blender = blend_models(estimator_list=[tuned_lgbm, tuned_rf])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,5.4591,50.7910,7.1268,0.8004,0.2193,0.1746
1,5.4809,51.2769,7.1608,0.7978,0.2202,0.1751
2,5.4989,51.4995,7.1763,0.7977,0.2204,0.1753
3,5.4576,50.5827,7.1121,0.8015,0.2183,0.1736
4,5.5103,51.8017,7.1973,0.7965,0.2209,0.1760
5,5.4765,51.1856,7.1544,0.7993,0.2193,0.1743
6,5.4662,50.8972,7.1342,0.7998,0.2192,0.1742
7,5.4745,50.9017,7.1345,0.7994,0.2179,0.1735
8,5.4615,50.7838,7.1263,0.8006,0.2198,0.1746


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,5.0918,51.7187,7.1916,0.7968,0.2151,0.1555
1,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
2,5.1093,52.0983,7.2179,0.7953,0.2155,0.1554
3,5.1185,52.3527,7.2355,0.7946,0.2157,0.1556
4,5.1214,52.3045,7.2322,0.7945,0.2156,0.1558
5,5.1039,52.1521,7.2216,0.7955,0.2152,0.1552
6,5.0874,51.7162,7.1914,0.7966,0.2143,0.1545
7,5.0909,51.7935,7.1968,0.7959,0.2142,0.1543
8,5.1073,52.2269,7.2268,0.7950,0.2159,0.1556


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.2313,34.0628,5.8363,0.8662,0.1788,0.1309
1,4.2312,34.0559,5.8357,0.8657,0.1784,0.1306
2,4.2312,34.1236,5.8415,0.8659,0.1785,0.1305
3,4.2555,34.3370,5.8598,0.8653,0.1787,0.1311
4,4.2487,34.3860,5.8640,0.8649,0.1791,0.1313
5,4.2423,34.3043,5.8570,0.8655,0.1783,0.1306
6,4.2171,33.8871,5.8213,0.8667,0.1774,0.1297
7,4.2316,34.0179,5.8325,0.8660,0.1772,0.1298
8,4.2061,33.7355,5.8082,0.8676,0.1779,0.1298


[I 2022-10-10 00:46:39,465] Searching the best hyperparameters using 3760973 samples...


[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9884585190307693
[LightGBM] [Warning] bagging_fraction is set=0.810897843150918, subsample=1.0 will be ignored. Current value: bagging_fraction=0.810897843150918
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9884585190307693
[LightGBM] [Warning] bagging_fraction is set=0.810897843150918, subsample=1.0 will be ignored. Current value: bagging_fraction=0.810897843150918
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9884585190307693
[LightGBM] [Warning] bagging_fra

[I 2022-10-10 04:16:35,853] Finished hyperparemeter search!


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:29:31
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2022-10-10 04:29:34,134] Searching the best hyperparameters using 3760973 samples...
[W 2022-10-10 04:29:34,195] The parameter 'actual_estimator__n_estimators' in trial#11 is sampled independently instead of being sampled by multivariate TPE sampler. (optimization performance may be degraded). You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `TPESampler`, if this independent sampling is intended behavior.
[W 2022-10-10 04:29:34,205] The parameter 'actual_estimator__n_estimators' in trial#12 is sampled independently instead of being sampled by multivariate TPE sampler. (optimization performance may be degraded). You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `TPESampler`, if this independent sampling is intended behavior.
[W 2022-10-10 04:29:34,214] The parameter 'actual_estimator__n_estimators' in trial#13 is sampled independently instead of being sampled by multivariate TPE s

In [ ]:
tuned_lgbm

LGBMRegressor(bagging_fraction=0.5047072983278198, bagging_freq=3, device='gpu',
              feature_fraction=0.6088578176376449,
              learning_rate=0.08365951934646977, min_child_samples=53,
              min_split_gain=0.8309351078973107, n_estimators=298,
              num_leaves=139, random_state=123, reg_alpha=0.617212302854929,
              reg_lambda=2.26630867328194e-06)

In [ ]:
tuned_rf

RandomForestRegressor(n_jobs=-1, random_state=123)

In [ ]:
blender = blend_models(estimator_list=[tuned_lgbm, tuned_rf])
final_model = finalize_model(blender)
prediction = predict_model(final_model, test)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.7332,41.6474,6.4535,0.8342,0.1975,0.1471
1,4.7314,41.6936,6.4571,0.8395,0.2004,0.1493
2,4.7219,41.6402,6.4529,0.8391,0.2013,0.1502
3,4.8528,44.1228,6.6425,0.8241,0.2030,0.1513
4,4.7695,44.7526,6.6897,0.8174,0.2021,0.1498
5,4.6603,40.3302,6.3506,0.8423,0.1933,0.1450
6,4.8029,43.0645,6.5624,0.8305,0.2030,0.1514
7,4.7957,42.3323,6.5063,0.8330,0.1998,0.1500
8,4.7682,42.6724,6.5324,0.8361,0.2020,0.1519


In [ ]:
pred = prediction['Label']

# 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv("./submit.csv", index = False)

sample_submission

,id,target
0,TEST_000000,23.26
1,TEST_000001,46.69
2,TEST_000002,62.56
3,TEST_000003,32.21
4,TEST_000004,39.54
...,...,...
291236,TEST_291236,43.78
291237,TEST_291237,51.56
291238,TEST_291238,35.75
291239,TEST_291239,23.96


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    scoring=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    scoring : str or callable, default=None
        A str (see model evaluation documentation) or
        a scorer callable object / function with signature
        ``scorer(estimator, X, y)``.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        scoring=scoring,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

X = train.drop(['target'], axis=1)
y = train['target']

title = "Learning Curves (Naive Bayes)"
# Cross validation with 50 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=axes[:, 0],
    ylim=(0.7, 1.01),
    cv=cv,
    n_jobs=4,
    scoring="accuracy",
)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = lgb.LGBMRegressor(random_state=42)
plot_learning_curve(
    estimator, title, X, y, axes=axes[:, 1], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)

plt.show()